### Import Required Package

In [35]:
import pandas as pd
import talib
import numpy as np
import json

### Import Tidal

In [36]:
import tidal as td

### Initialize NutrLink

In [37]:
from nutrlink import NutrLink
from nutrlink.helper import get_ohlcv
nl = NutrLink(url="https://dev-api.ddt-dst.cc/nutrients/station")

### Load market data and compute technical indicators

In [38]:
import os
import pandas as pd

# os.chdir('/home/jovyan/individualized-indicator')
# 定義你想要讀取檔案的目錄
directory = '../emb/spearman'

start_date = "2022-06-01"
end_date = "2024-09-30"

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        file_path = os.path.join(directory, filename)
        # 讀取 .pkl 檔案
        df = pd.read_pickle(file_path)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = get_ohlcv(
        nl,
        start = start_date,
        end = end_date,
        tickers = [stock for stock in sorted_list],
)

# reset index，把 MultiIndex 攤平成兩個欄位
df = total_data.reset_index()

# 把 datetime 欄位轉成你要的格式
df["datetime"] = pd.to_datetime(df["datetime"]).dt.strftime("%Y-%m-%d")

# 再把 ticker, datetime 設回 MultiIndex
total_data = df.set_index(["ticker", "datetime"])

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+1)%12, (select_mon+2)%12, (select_mon+3)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
        
    if select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'

# quote_data.to_csv('./quote_data.csv')

### Load benchmark data

In [39]:
benchmark_inst = "0050"
benchmark_data = get_ohlcv(
        nl,
        start = start_date,
        end = end_date,
        tickers = [benchmark_inst],
)

# reset index，把 MultiIndex 攤平成兩個欄位
df = benchmark_data.reset_index()

# 把 datetime 欄位轉成你要的格式
df["datetime"] = pd.to_datetime(df["datetime"]).dt.strftime("%Y-%m-%d")

# 再把 ticker, datetime 設回 MultiIndex
benchmark_data = df.set_index(["ticker", "datetime"])

benchmark_data.index = benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),level=1)

benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

open     high      low    close    volume
instrument datetime                                                
0050       2022-06-01  29.2074  29.3769  29.1397  29.1736   6730117
           2022-06-02  29.0154  29.0154  28.8234  28.8686   4721923
           2022-06-06  28.9703  29.1397  28.7896  29.0719   4292361
           2022-06-07  28.9025  28.9025  28.6540  28.7783   5160242
           2022-06-08  29.0267  29.1623  28.9477  29.0945   4110313
...                        ...      ...      ...      ...       ...
           2024-09-24  44.8572  45.2979  44.5879  45.2979  10476465
           2024-09-25  45.9591  46.0325  45.8734  45.9713  12464313
           2024-09-26  46.4243  46.5222  46.2161  46.3508  17369543
           2024-09-27  46.5712  46.8283  46.1794  46.2529  11999510
           2024-09-30  45.6775  45.8366  45.0409  45.0409  14461065

[569 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [40]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=50000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi", ignore_volume_size=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(15, 1, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

[2025/09/15 16:31:07] root INFO Allocated remote server address: tcp://10.136.20.6:6666
[2025/09/15 16:31:07] root INFO SocketClient Initializing connection to tcp://10.136.20.6:6666
[2025/09/15 16:31:07] root INFO SocketClient tcp://10.136.20.6:6666: Starting event monitor
[2025/09/15 16:31:07] root INFO SocketClient tcp://10.136.20.6:6666: Attempting handshake
[2025/09/15 16:31:11] root INFO SocketClient tcp://10.136.20.6:6666: Successfully connected to server
[2025/09/15 16:31:11] root INFO SocketClient tcp://10.136.20.6:6666: ZMQ connection established, waiting for server response
[2025/09/15 16:31:11] root INFO SocketClient tcp://10.136.20.6:6666: Handshake completed successfully
[2025/09/15 16:31:11] root INFO SocketClient Successfully initialized connection to tcp://10.136.20.6:6666
[2025/09/15 16:31:11] root INFO Tidal client version: 1.1.70
[2025/09/15 16:31:11] root INFO Tidal server version: 1.1.70


### Stock Config

In [42]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [43]:
tidal.exchange.inst_configs

{'NQ': InstConfig {Margin:33613.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'TX': InstConfig {Margin:357000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:89250.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05}}

### Config Modification

In [44]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [45]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'NQ': InstConfig {Margin:33613.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'TX': InstConfig {Margin:357000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:89250.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [46]:
tidal.backtest()
logs = tidal.get_logs()
for log in logs:
    print(log)

Tidal Backtesting: 100%|██████████| 548/548 [01:33<00:00,  5.88it/s, cash=1.99e+5, pnl=-9.57e+5, position_cost=7.9e+7, value=7.83e+7] 

2022-10-03 - 訂單未能執行: 6245, 可能是由於現金不足或其他原因
2023-09-26 - 訂單未能執行: 2404, 可能是由於現金不足或其他原因


### Metric - AccountInfo

In [ ]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

### Metric - PositionInfo

In [ ]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

In [ ]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

### Metric - AdditionalInfo

In [ ]:
tidal.metrics['AdditionalInfo'].report

### Strategy Lake Submit

In [ ]:
# group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
# submit_lake_backtest_result = tidal.submit_lake_backtest(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
# print(submit_lake_backtest_result)

### Trade Report

In [ ]:
tidal.trade_report

### Traded instruments

In [ ]:
tidal.account.trades.keys()

### Trading History

In [ ]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [ ]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [47]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


[2025/09/15 16:34:12] werkzeug INFO WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:40055
 * Running on http://10.136.39.77:40055
[2025/09/15 16:34:12] werkzeug INFO Press CTRL+C to quit


[2025/09/15 16:34:13] werkzeug INFO 10.0.10.82 - - [15/Sep/2025 16:34:13] "GET / HTTP/1.1" 200 -
[2025/09/15 16:34:14] werkzeug INFO 10.0.10.82 - - [15/Sep/2025 16:34:14] "GET /static/js/main.5993c177.js HTTP/1.1" 200 -
[2025/09/15 16:34:15] werkzeug INFO 10.0.10.82 - - [15/Sep/2025 16:34:15] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
[2025/09/15 16:34:15] werkzeug INFO 10.0.10.82 - - [15/Sep/2025 16:34:15] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
[2025/09/15 16:34:15] werkzeug INFO 10.0.10.82 - - [15/Sep/2025 16:34:15] "GET /api/quote/inst_list HTTP/1.1" 200 -
[2025/09/15 16:34:15] werkzeug INFO 10.0.10.82 - - [15/Sep/2025 16:34:15] "GET /api/metric/metric_list HTTP/1.1" 200 -
[2025/09/15 16:34:16] werkzeug INFO 10.0.10.82 - - [15/Sep/2025 16:34:16] "GET /api/trade/trade_report HTTP/1.1" 200 -
[2025/09/15 16:34:16] werkzeug INFO 10.0.10.82 - - [15/Sep/2025 16:34:16] "GET /Tidal_Logo.png HTTP/1.1" 200 -
[2025/09/15 16:34:16] werkzeug INFO 10.0.10.82 - - [15/Sep/2025 16

In [ ]:
tidal.account.position_history

In [ ]:
Portfolio_info = tidal.metrics["Portfolio"].report
print(Portfolio_info)

In [ ]:
Portfolio_info.to_csv('./Portfolio_info.csv')